# Script to process data for ML model

In [1]:
# import the necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import pickle
import warnings
import geopandas as gpd
from shapely.geometry import Point
from pyproj import CRS
import pickle

## Calculate the monthly average for all stations

In [2]:
# import the previously saved stations data to the notebook

NO3_N_dfs_filtered = None
with open("NO3_N_dfs_filtered", 'rb') as file:
    # Deserialize and retrieve the variable from the file
    NO3_N_dfs_filtered = pickle.load(file)

In [3]:
thresh = 2         # data for minimum thresh years only considered
edited_dataframes_list = [df for i,df in enumerate(NO3_N_dfs_filtered) if len(df.iloc[:,1].unique()) > thresh]
print("Number of stations after applying minimum years threshold = ", len(edited_dataframes_list))

Number of stations after applying minimum years threshold =  500


In [4]:
#average data according to month and then create individual dataframes for each month. Because at the end we will have different models for each month.


# average the data according to month. Some stations might not have any data for a particular month.
avg_dfs = []
for i, df in enumerate(edited_dataframes_list):
    monthly_avg_df = df.groupby('month', as_index=False).agg({
        'NO3-N [mg/l]': 'mean',
        'NO3-N [mg/l] BC': 'mean',
        'easting': 'first',  # Assuming the same for all entries
        'northing': 'first',  # Assuming the same for all entries
        'station_name': 'first',  # Assuming the same for all entries
        'station_number': 'first',  # Assuming the same for all entries
        'water_body': 'first'  # Assuming the same for all entries
    })
    avg_dfs.append(monthly_avg_df)
    
# Stack all the dataframes vertically to compute.  
combined_avg_dfs = pd.concat(avg_dfs, axis=0)

In [5]:
combined_avg_dfs

,month,NO3-N [mg/l],NO3-N [mg/l] BC,easting,northing,station_name,station_number,water_body
0,1,6.162333,2.671989,693538.0,5549278.0,"Strbr. in Höhe v. Stein, obh. KA",13516,Ölschnitz
1,2,6.493667,2.781729,693538.0,5549278.0,"Strbr. in Höhe v. Stein, obh. KA",13516,Ölschnitz
2,3,6.200000,2.684981,693538.0,5549278.0,"Strbr. in Höhe v. Stein, obh. KA",13516,Ölschnitz
3,4,5.755333,2.533111,693538.0,5549278.0,"Strbr. in Höhe v. Stein, obh. KA",13516,Ölschnitz
4,5,5.348667,2.387275,693538.0,5549278.0,"Strbr. in Höhe v. Stein, obh. KA",13516,Ölschnitz
...,...,...,...,...,...,...,...,...
7,8,0.610800,-0.453432,643380.0,5356423.0,Augsburg Hochablaß,3044,Lech
8,9,0.654850,-0.398177,643380.0,5356423.0,Augsburg Hochablaß,3044,Lech
9,10,0.752943,-0.279284,643380.0,5356423.0,Augsburg Hochablaß,3044,Lech
10,11,0.897564,-0.122637,643380.0,5356423.0,Augsburg Hochablaß,3044,Lech


## Import hydro-environmental variables

In [6]:
# function to load environmental variables to the notebook. These datasets were processed and saved in other notebooks. 

def import_features(feature):
    clim_var = None
    with open(feature, 'rb') as file:
        # Deserialize and retrieve the variable from the file
        clim_var = pickle.load(file)
    return clim_var

In [7]:
# Implement the import_features function to load environmental variables

df_filtered_slope = import_features("df_filtered_slope")
df_filtered_elev_avg = import_features("df_filtered_elev_avg")
df_filtered_landcover = import_features("df_filtered_landcover")
df_filtered_hydroclim = import_features("df_filtered_hydroclim")
df_filtered_soil_avg = import_features("df_filtered_soil_avg")
df_filtered_tmin_avg = import_features("df_filtered_tmin_avg")
df_filtered_tmax_avg = import_features("df_filtered_tmax_avg")
df_filtered_prec_us_sum = import_features("df_filtered_prec_us_sum")

In [8]:
df_filtered_slope

variable,index,lat,lon,3
0,1902,47.295833,10.229167,1423.0
1,2487,47.304167,10.237500,1436.0
2,2488,47.304167,10.245833,1463.0
3,3073,47.312500,10.254167,1493.0
4,3074,47.312500,10.262500,1532.0
...,...,...,...,...
16445,228480,50.529167,10.112500,350.0
16446,228481,50.529167,10.120833,370.0
16447,228482,50.529167,10.129167,373.0
16448,228484,50.529167,10.145833,366.0


In [9]:
df_filtered_landcover

variable,index,lat,lon,0,1,2,3,4,5,6,7,8,9,10,11
0,1902,47.295833,10.229167,29.0,0.0,6.0,7.0,3.0,25.0,20.0,0.0,0.0,0.0,8.0,2.0
1,2487,47.304167,10.237500,28.0,0.0,6.0,6.0,3.0,25.0,21.0,0.0,0.0,0.0,8.0,2.0
2,2488,47.304167,10.245833,28.0,0.0,8.0,8.0,4.0,23.0,20.0,0.0,0.0,0.0,7.0,2.0
3,3073,47.312500,10.254167,27.0,0.0,7.0,9.0,4.0,24.0,21.0,0.0,0.0,0.0,7.0,2.0
4,3074,47.312500,10.262500,25.0,0.0,7.0,10.0,5.0,23.0,18.0,0.0,0.0,0.0,10.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16445,228480,50.529167,10.112500,0.0,0.0,7.0,10.0,0.0,10.0,66.0,0.0,0.0,0.0,7.0,0.0
16446,228481,50.529167,10.120833,0.0,0.0,7.0,9.0,0.0,10.0,66.0,0.0,0.0,0.0,9.0,0.0
16447,228482,50.529167,10.129167,0.0,0.0,6.0,8.0,0.0,9.0,66.0,0.0,0.0,0.0,11.0,0.0
16448,228484,50.529167,10.145833,0.0,0.0,10.0,17.0,0.0,5.0,55.0,0.0,0.0,0.0,12.0,0.0


In [10]:
df_filtered_tmin_avg

variable,index,lat,lon,0,1,2,3,4,5,6,7,8,9,10,11
0,1902,47.295833,10.229167,-79.0,-72.0,-40.0,-3.0,36.0,69.0,88.0,85.0,62.0,21.0,-24.0,-62.0
1,2487,47.304167,10.237500,-81.0,-74.0,-43.0,-7.0,33.0,66.0,85.0,82.0,60.0,19.0,-26.0,-64.0
2,2488,47.304167,10.245833,-80.0,-73.0,-42.0,-6.0,33.0,66.0,86.0,83.0,60.0,20.0,-25.0,-64.0
3,3073,47.312500,10.254167,-80.0,-74.0,-43.0,-7.0,33.0,65.0,85.0,82.0,59.0,19.0,-26.0,-64.0
4,3074,47.312500,10.262500,-80.0,-73.0,-42.0,-6.0,34.0,66.0,86.0,83.0,60.0,20.0,-25.0,-64.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16445,228480,50.529167,10.112500,-48.0,-43.0,-18.0,10.0,51.0,84.0,102.0,99.0,67.0,32.0,0.0,-27.0
16446,228481,50.529167,10.120833,-47.0,-43.0,-18.0,10.0,52.0,85.0,102.0,100.0,68.0,32.0,0.0,-27.0
16447,228482,50.529167,10.129167,-47.0,-42.0,-17.0,10.0,52.0,85.0,102.0,100.0,68.0,32.0,0.0,-26.0
16448,228484,50.529167,10.145833,-46.0,-41.0,-16.0,11.0,53.0,86.0,103.0,101.0,69.0,33.0,1.0,-25.0


## Process dataframe in required format

In [33]:
def process_dataframe(month_num, month_name, df_landcover, df_filtered_slope, df_filtered_elev_avg, df_filtered_landcover, df_filtered_hydroclim, df_filtered_soil_avg, df_filtered_tmin_avg, df_filtered_tmax_avg, df_filtered_prec_us_sum):
    """
    Computes the average monthly cconcentration for stations for every momth and formats the dataframes in required format. Saves the monthly dataframes to disk.
    Args:
        month_num (pandas dataframe): number of month. 1 for january, 12 for december
        month_name (string): name of the month to add to the saved variable
    """
    
    month_avg_concs = combined_avg_dfs[combined_avg_dfs.iloc[:,0] == month_num].reset_index(drop = True)
    
    ### SNAP STATIONS TO STREAM NETWORK
    
    month_avg_conc_new_loc = month_avg_concs.copy()
    month_avg_conc_new_loc['closest_lon'] = ""
    month_avg_conc_new_loc['closest_lat'] = ""
    # Converting coordinates of the df_filtered_landcover to a geodataframe geometry for easier processing. Changing CRS to calculate distance of points in meters. 
    df_landcover_new = gpd.GeoDataFrame( df_filtered_landcover, geometry=gpd.points_from_xy(df_filtered_landcover.lon, df_filtered_landcover.lat),crs=CRS('EPSG:4326')).to_crs(epsg=25832)

    # for each station, find the closest point in the stream network. Assign the station the coordinates of this closest point. If a station already has been asssigned this point, assign the next closest point
    assigned_points = []
    assigned_distances = []
    for i in range(month_avg_conc_new_loc.shape[0]):
        station_point = Point(month_avg_conc_new_loc.iloc[i,3], month_avg_conc_new_loc.iloc[i,4] )
        distances =  df_landcover_new.distance(station_point)
        sorted_distances = distances.sort_values()
        
        for idx in sorted_distances.index:

            # If the pixel in the stream network is already associated with a station, choose the next closest pixel
            if idx not in assigned_points:
                min_index = idx
                assigned_points.append(idx)
                assigned_distances.append(sorted_distances[idx])
                break
        
        month_avg_conc_new_loc.iloc[i,8] = df_landcover_new.iloc[min_index, 2]
        month_avg_conc_new_loc.iloc[i,9] = df_landcover_new.iloc[min_index, 1]
    
    
    ### MERGE ALL THE DATA REQUIRED TO RUN THE MODEL IN A SINGLE DATAFRAME
    
    # Merge the dataframe one by one using the coorindates as the common data. Drop some columns to improve formatting. 
    # Suffixes option is specified while merging because all environmental variables dataframes have similar naming convention for columns
    df3 = pd.merge(month_avg_conc_new_loc, df_filtered_slope, left_on=['closest_lon', 'closest_lat'], right_on=['lon', 'lat'], how='left')
    df3 = df3.drop(columns=['closest_lon', 'closest_lat', 'month','easting','northing','index'])
    df4 = pd.merge(df3, df_filtered_elev_avg, on=['lon', 'lat'], how='left')
    df4 = df4.drop(columns='index')
    df5 = pd.merge(df4, df_filtered_landcover, on=['lon', 'lat'], how='left')
    df5 = df5.drop(columns='index')
    df6 = pd.merge(df5, df_filtered_hydroclim, on=['lon', 'lat'], how='left',suffixes=(None, 'hydroclim'))
    df6 = df6.drop(columns='index')
    df7 = pd.merge(df6, df_filtered_soil_avg, on=['lon', 'lat'], how='left',suffixes=(None, 'soil_avg'))
    df7 = df7.drop(columns='index')
    df8 = pd.merge(df7, df_filtered_tmin_avg.iloc[:,[0,1,2] + [2+month_num]], on=['lon', 'lat'], how='left',suffixes=(None, 'tmin_avg'))
    df8 = df8.drop(columns='index')
    df9 = pd.merge(df8, df_filtered_tmax_avg.iloc[:,[0,1,2] + [2+month_num]], on=['lon', 'lat'], how='left',suffixes=(None, 'tmax_avg'))
    df9 = df9.drop(columns='index')
    final_df_month_new = pd.merge(df9, df_filtered_prec_us_sum.iloc[:,[0,1,2] + [2+month_num]], on=['lon', 'lat'], how='left',suffixes=(None, 'prec_us_sum'))
    final_df_month_new  = final_df_month_new.drop(columns='index')
    
    # Change the names of the columns for better readability 
    new_names = ['slope_avg',
    'elev_avg',
    'lc_avg_01',
    'lc_avg_02',
    'lc_avg_03',
    'lc_avg_04',
    'lc_avg_05',
    'lc_avg_06',
    'lc_avg_07',
    'lc_avg_08',
    'lc_avg_09',
    'lc_avg_10',
    'lc_avg_11',
    'lc_avg_12',
    'hydro_avg_01',
    'hydro_avg_02',
    'hydro_avg_03',
    'hydro_avg_04',
    'hydro_avg_05',
    'hydro_avg_06',
    'hydro_avg_07',
    'hydro_avg_08',
    'hydro_avg_09',
    'hydro_avg_10',
    'hydro_avg_11',
    'hydro_avg_12',
    'hydro_avg_13',
    'hydro_avg_14',
    'hydro_avg_15',
    'hydro_avg_16',
    'hydro_avg_17',
    'hydro_avg_18',
    'hydro_avg_19',
    'soil_avg_01',
    'soil_avg_02',
    'soil_avg_03',
    'soil_avg_04',
    'soil_avg_05',
    'soil_avg_06',
    'soil_avg_07',
    'soil_avg_08',
    'soil_avg_09',
    'soil_avg_10',
    f'tmin_avg_{month_num}',
    f'tmax_avg_{month_num}',
    f'prec_sum_{month_num}',
    ]
    
    # Create a dictionary mapping old column names to new ones
    rename_dict = {final_df_month_new.columns[i]: new_names[i - 7] for i in range(7, 53)}
    
    # Rename the columns using the names defined above
    final_df_month_new.rename(columns=rename_dict, inplace=True)
    
    
    # save the filtered dataframe
    filepath = f"final_df_{month_name}_new"
    with open(filepath, 'wb') as file:
        pickle.dump(final_df_month_new, file)

    return assigned_distances

In [34]:
month_names = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]
month_num = list(range(1,13))
distances_months = []
# Save the dataframe for each month to disk
for i in range(1,13):
    print('Currently processcing data for month: ', month_names[i-1])
    dist = process_dataframe(i, month_names[i-1], df_filtered_landcover, df_filtered_slope, df_filtered_elev_avg, df_filtered_landcover, df_filtered_hydroclim, df_filtered_soil_avg, df_filtered_tmin_avg, df_filtered_tmax_avg, df_filtered_prec_us_sum)
    distances_months.append(dist)

Currently processcing data for month:  jan
Currently processcing data for month:  feb
Currently processcing data for month:  mar
Currently processcing data for month:  apr
Currently processcing data for month:  may
Currently processcing data for month:  jun
Currently processcing data for month:  jul
Currently processcing data for month:  aug
Currently processcing data for month:  sep
Currently processcing data for month:  oct
Currently processcing data for month:  nov
Currently processcing data for month:  dec


In [37]:
monthly_distances_avg = []
for dist_month in distances_months:
    avg_dist_month = sum(dist_month) / len(dist_month)
    monthly_distances_avg.append(avg_dist_month)

In [39]:
sum(monthly_distances_avg)/len(monthly_distances_avg)

546.5616264554429